# define the COBRA model

In [1]:
# define the environment path 
import os
local_cobrakbase_path = os.path.join('C:', 'Users', 'Andrew Freiburger','Documents','Argonne','cobrakbase')
os.environ["HOME"] = local_cobrakbase_path

import cobrakbase
token = 'MIXOUVGIFOUPQD2KSH6DQ5ARKDLBARCL'
kbase = cobrakbase.KBaseAPI(token)

model = kbase.get_from_ws("CMM_iAH991V2_iML1515.kb",40576)
media = kbase.get_from_ws("Btheta_Ecoli_minimal_media",40576)

cobrakbase 0.2.8


# define Python execution function

In [9]:
def execute_julia(cobra_model):
    # setup Julia
    from julia.api import Julia
    jl = Julia(compiled_modules=False)
    from julia import Main
    # jl.using("TextAnalysis")
    jl.eval('include("cobra_execution.jl")')
    
    # load the model
    from cobra.io import save_matlab_model
    model_path = 'test.mat'
    save_matlab_model(model, model_path)
    Main.model_path = model_path
    solution = jl.eval("execute_cobra(model_path)")
    
    # remove the MATLAB file
    from os import remove
    remove(model_path)
    
    return solution

## Python time

In [3]:
%time python_sol = model.optimize()

CPU times: total: 484 ms
Wall time: 476 ms


## Julia time

In [12]:
%time julia_sol = execute_julia(model)

# execute_cobra('test.mat')

JuliaError: Exception 'The solver `GLPK` or `GLPKMathProgInterface` cannot be set using `changeCobraSolver()`.' occurred while calling julia code:
execute_cobra(model_path)